# **Create Single Monkey Surfaces**  

This notebook will guide you through the procedure to create single monkey surface representations in four steps (some are optional):  

1. **(OPTIONAL)** Average multiple T1 scans from the same animal. (takes seconds)
2. Segment the (averaged) T1 of an individual animal using the NMT template as a prior. (takes many hours)
3. Create surfaces and flatmaps using Freesurfer. (takes minutes to hours depending on segmentation quality at the start)
4. **(OPTIONAL)** Create additional surfaces with Connectome Workbench. (takes minutes)

## Initiate 
Setting a few environment variables here so we can actually run code from this notebook

In [1]:
SUBJ=Danny  # Name of the animal you are going to process
NMT_path=/NHP_MRI/Template/NMT # path to the template folder
startpath=pwd 

## **Step 1: Average multiple T1 scans**
### Requirements:
- Freesurfer installation (https://surfer.nmr.mgh.harvard.edu/)
- NIH Macaque Template (https://github.com/jms290/NMT)
- One or more high resolution anatomical scans

- The `average_multiple_t1.sh ` script  
  

### Procedure
- Create a folder for your subject in the NIH Macaque Template folder as 

`<>/NHP_MRI/Template/NMT/single_subject_scans/<SUBJECT>`  

- Inside this folder, create a subfolder with all individual T1 scan for this subject, e.g. 

`<>/NHP_MRI/Template/NMT/single_subject_scans/<SUBJECT>/<T1_FLD>/*.nii.gz`  

- Copy the script `average_multiple_t1.sh` to the `<SUBJECT>` folder as well.  

- Run this script with the name of the `<T1_FLD>` as input:  

`$ ./average_multiple_t1.sh <T1_FLD>`  


- The script will:  

    - Correct for having the monkey in the sphinx position  
    - Resample each T1 to 0.5 mm isotropic voxels  
    - Reorient the colume for correct display in FSL (e.g., with FSLEyes)  
    - Normalize the contrast gradient  
    - Average the multiple T1's together using motion correction to account for small differences  
  
  
- When the script is done, there should be a `<SUBJECT>.nii.gz` file in the root `<SUBJECT>` folder.

In [ ]:
NMT_ss_path=${NMT_path}/single_subject_scans/${SUBJ}
mkdir -p ${NMT_ss_path}
mkdir -p ${NMT_path}/single_subject_scans/${SUBJ}/T1s
echo Copy the T1 files to ${NMT_path}/single_subject_scans/${SUBJ}/T1s

In [ ]:
cp ${NMT_path}/single_subject_scans/average_multiple_t1.sh \
    ${NMT_path}/single_subject_scans/${SUBJ}/
cd ${NMT_path}/single_subject_scans/${SUBJ}/
./average_multiple_t1.sh T1s

## **Step 2: Segment the single subject anatomy using the NMT template as a prior**
### Requirements
  - NIH Macaque Template (https://github.com/jms290/NMT)  
  - ANTS (http://stnava.github.io/ANTs/)
  - AFNI (https://afni.nimh.nih.gov/)
 
### Procedure
This will segment and process the single-subject anatomical and warp the D99 atlas (https://afni.nimh.nih.gov/Macaque) to your individual anatomy.  

- Navigate to `<>/NHP_MRI/Template/NMT/single_subject_scans` in your terminal
  
- There should be a folder `<SUBJECT>` containing a T1 named `<SUBJECT>.nii.gz` (Step 1 produces this)
  
- Run the `align_and_process_singlesubject.sh` script with input `<SUBJECT>`:
      
- The following 2 steps can be executed together by running: `$ ./create_ss_atlas.sh <SUBJECT>`  

    - Extracting single ROI masks from the warped atlas: `./extract_ROI.sh <SUBJECT>`  

    - Creating a label-file that makes the warped atlas usable in FSLEyes: `./create_xml.sh <SUBJECT>`


In [ ]:
cd ${NMT_path}/single_subject_scans
./align_and_process_singlesubject.sh ${SUBJ}
./create_ss_atlas ${SUBJ}

## **Step 3: Create surfaces and flatmaps using Freesurfer**

### Requirements  
- Freesurfer installation (https://surfer.nmr.mgh.harvard.edu/)
- Segmented individual subject T1

### Procedure
#### Create folders & copy files

In [ ]:
#NMT_src=${NMT_path}/single_subject_scans/${SUBJ}/NMT_${SUBJ}_process 
NMT_src=${NMT_path}/single_subject_scans/${SUBJ}/NMT_${SUBJ}_process/manual_adjust # danny 
# adjust this if you have manually adjusted and saved to a different folder

fsSurf=${NMT_path}/single_subject_scans/${SUBJ}/fsSurf
mkdir -p ${fsSurf}

fsSurf_src=${fsSurf}/src
fsSurf_mgz=${fsSurf}/mgz
fsSurf_temp=${fsSurf}/temp
mkdir -p ${fsSurf_src}
mkdir -p ${fsSurf_mgz}
mkdir -p ${fsSurf_temp}

# source files
cp ${NMT_src}/${SUBJ}_N4.nii.gz ${fsSurf_src}/T1.nii.gz
cp ${NMT_src}/${SUBJ}_brain.nii.gz ${fsSurf_src}/brain.nii.gz
cp ${NMT_src}/${SUBJ}_brain.nii.gz ${fsSurf_src}/brainmask.nii.gz # freesurfer brainmask aren't actually masks
cp ${NMT_src}/${SUBJ}_segmentation_WM.nii.gz ${fsSurf_src}/wm.nii.gz

# copy source files again but keep these original (for others the header will be changed to 'fake' 1 mm voxels)
mkdir -p ${fsSurf_src}/org
cp ${fsSurf_src}/*.nii.gz ${fsSurf_src}/org/

#### Change headers to mimic 1mm isotropic voxels & make mgz files

In [ ]:
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/T1.nii.gz
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/brain.nii.gz
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/brainmask.nii.gz
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/wm.nii.gz

mri_convert -c ${fsSurf_src}/T1.nii.gz ${fsSurf_mgz}/T1.mgz
mri_convert -c ${fsSurf_src}/brain.nii.gz ${fsSurf_mgz}/brain.mgz
mri_convert -c ${fsSurf_src}/brainmask.nii.gz ${fsSurf_mgz}/brainmask.mgz
mri_convert -c ${fsSurf_src}/wm.nii.gz ${fsSurf_mgz}/wm.mgz

mri_mask -T 5 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/brainmask.mgz ${fsSurf_mgz}/brain.finalsurfs.mgz

#### Get corpus callosum and pons voxel coordinates
Corpus callosum:  
![CC](pics/CC_coordinates.png)

Pons: 
![PONS](pics/PONS_coordinates.png)

In [ ]:
# Inspect volume to get voxel coordinates
freeview -v ${fsSurf_mgz}/brain.mgz &

In [ ]:
CC=(127 114 129)
PONS=(124 148 119)

#### Fill the white matter volume & copy for fixing

In [ ]:
# Fill WM
mri_fill -CV ${CC[0]} ${CC[1]} ${CC[2]} \
    -PV ${PONS[0]} ${PONS[1]} ${PONS[2]} \
    ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/filled.mgz
    
cp ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/wm_nofix.mgz

#### Tesselate volumes & fix topology (First run)

In [ ]:
# left
mri_pretess ${fsSurf_mgz}/filled.mgz 255 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess255.mgz
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess255.mgz 255 ${fsSurf_temp}/lh.orig.nofix
# right
mri_pretess ${fsSurf_mgz}/filled.mgz 127 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess127.mgz
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess127.mgz 127 ${fsSurf_temp}/rh.orig.nofix

HEMI=(lh rh) # array to loop over hemispheres
for xh in ${HEMI[@]}; do
    cp ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.orig

    # post-process tesselation
    mris_extract_main_component ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.orig.nofix
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.smoothwm.nofix
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm.nofix ${fsSurf_temp}/${xh}.inflated.nofix
    mris_sphere -q ${fsSurf_temp}/${xh}.inflated.nofix ${fsSurf_temp}/${xh}.qsphere.nofix
    cp ${fsSurf_temp}/${xh}.inflated.nofix ${fsSurf_temp}/${xh}.inflated

    # fix topology
    mris_euler_number ${fsSurf_temp}/${xh}.orig
    mris_remove_intersection ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
done

#### Iterate manual adjustments and doing tesselation & topology fix again
Should look something like this. Keep going (adjust wm.mgz using the recon-edit function of Freeview) until you are happy with the result.

![LH_wm_inflated](pics/LH_wm_inflated.png) ![RH_wm_inflated](pics/RH_in_wm_inflated.png)


In [ ]:
# look at the result and apply fixes in WM definition
freeview -v ${fsSurf_mgz}/brain.mgz -v ${fsSurf_mgz}/wm.mgz \
    -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated \
    ${fsSurf_temp}/rh.smoothwm ${fsSurf_temp}/rh.inflated &

In [ ]:
# redo the tesselation with the fixed WM volume
mri_fill -CV ${CC[0]} ${CC[1]} ${CC[2]} \
    -PV ${PONS[0]} ${PONS[1]} ${PONS[2]} \
    ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/filled.mgz
    
mri_pretess ${fsSurf_mgz}/filled.mgz 255 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess255.mgz
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess255.mgz 255 ${fsSurf_temp}/lh.orig
mri_pretess ${fsSurf_mgz}/filled.mgz 127 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess127.mgz
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess127.mgz 127 ${fsSurf_temp}/rh.orig

for xh in ${HEMI[@]}; do
    mris_extract_main_component ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
    mris_sphere -q ${fsSurf_temp}/${xh}.inflated ${fsSurf_temp}/${xh}.qsphere
    
    mris_euler_number ${fsSurf_temp}/${xh}.orig
    mris_remove_intersection ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
    mris_curvature -thresh .999 -n -a 5 -w -distances 10 10 ${fsSurf_temp}/${xh}.inflated
    mris_sphere ${fsSurf_temp}/${xh}.inflated ${fsSurf_temp}/${xh}.sphere
done
echo 'TESSELATION DONE!'

In [ ]:
# create sphere surface
for xh in ${HEMI[@]}; do
    mris_sphere ${fsSurf_temp}/${xh}.inflated ${fsSurf_temp}/${xh}.sphere
done

#### Create a subject directory in your freesurfer subject directory

In [ ]:
# create a freesurfer SUBJECT with the correct folder structure
# NB! $SUBJECTS_DIR should be defined in your ~/.bashrc as the Freesurfer subjects directory
echo 'Creating a subject directory for '${SUBJ}'in:'
echo ${SUBJECTS_DIR}/${SUBJ}
mksubjdirs ${SUBJECTS_DIR}/${SUBJ}

#### Cut surfaces and create flatmaps

In [ ]:
# copy the necessary files for cutting to the subject directory
for xh in ${HEMI[@]}; do
    cp ${fsSurf_temp}/${xh}.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.smoothwm ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.orig ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.qsphere ${SUBJECTS_DIR}/${SUBJ}/surf/
done

cp ${fsSurf_mgz}/T1.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/T1.mgz
cp ${fsSurf_mgz}/filled.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/filled.mgz
cp ${fsSurf_mgz}/wm.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz
cp ${fsSurf_mgz}/brain.finalsurfs.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brain.finalsurfs.mgz
cp ${fsSurf_mgz}/brainmask.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz

In [7]:
mris_make_surfaces -noaseg -noaparc -T1 brain -orig_white orig -orig_pial orig ${SUBJ} lh

not using aseg volume to prevent surfaces crossing the midline
not using aparc to prevent surfaces crossing the midline
using brain as T1 volume...
using orig starting white location...
using orig starting pial locations...
$Id: mris_make_surfaces.c,v 1.164.2.4 2016/12/13 22:26:32 zkaufman Exp $
$Id: mrisurf.c,v 1.781.2.6 2016/12/27 16:47:14 zkaufman Exp $
reading volume /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/filled.mgz...
reading volume /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/brain.mgz...
resampling filled volume to be in voxel register with T1
reading volume /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/wm.mgz...
resampling wm volume to be in voxel register with T1
315762 bright wm thresholded.
525282 bright non-wm voxels segmented.
reading original surface position from /Users/chris/Documents/MRI/freesurfer/subjects//Danny/surf/lh.orig...
computing class statistics...
border white:    159857 voxels (0.95%)
border gray      171366 voxe

In [5]:
# create surfaces
for xh in ${HEMI[@]}; do
    mris_make_surfaces -noaseg -noaparc -T1 T1 -orig_wm orig ${SUBJ} ${xh}
    mris_sphere ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.sphere
    mris_expand -thickness ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white 0.5 ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid
done

not using aseg volume to prevent surfaces crossing the midline
not using aparc to prevent surfaces crossing the midline
using T1 as T1 volume...
reading original vertex positions from orig
$Id: mris_make_surfaces.c,v 1.164.2.4 2016/12/13 22:26:32 zkaufman Exp $
$Id: mrisurf.c,v 1.781.2.6 2016/12/27 16:47:14 zkaufman Exp $
reading volume /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/filled.mgz...
reading volume /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/T1.mgz...
resampling filled volume to be in voxel register with T1
reading volume /Users/chris/Documents/MRI/freesurfer/subjects//Danny/mri/wm.mgz...
resampling wm volume to be in voxel register with T1
315613 bright wm thresholded.
916341 bright non-wm voxels segmented.
reading original surface position from /Users/chris/Documents/MRI/freesurfer/subjects//Danny/surf/lh.orig...
computing class statistics...
border white:    159857 voxels (0.95%)
border gray      171366 voxels (1.02%)
WM (109.0): 110.0 +- 0.2 [9

In [ ]:
# create surfaces
HEMI=(lh rh)
for xh in ${HEMI[@]}; do
    mris_make_surfaces -noaseg -noaparc -T1 brain.finalsurfs -orig_wm orig ${SUBJ} ${xh}
done

The cells below will load the inflated hemispheres in tksurfer where you can make the cuts to create the flatmaps.  

For the full hemisphere make cuts on the medial wall. One cut enclosing the midline and five additional relaxation cuts.
Mark a point on the part of the surface you want to keep and save patch as `?h.full.patch.3d` (replace '?' with 'l' and 'r' respectively)

![Cut_full](pics/Cut_full.png)

For an occipital patch make one cut on the medial wall along the calcarine sulcus. Use 3 points to select a coronal cutting plane, and a fourth point to select which part of the surface you want to keep and save as `?h.occip.patch.3d`

![Cut_occip1](pics/Cut_occip1.png) ![Cut_occip1](pics/Cut_occip2.png)

In [ ]:
# left
tksurfer ${SUBJ} lh inflated -curv

In [ ]:
# right
tksurfer ${SUBJ} rh inflated -curv

In [ ]:
cd ${SUBJECTS_DIR}/${SUBJ}/surf/
for xh in ${HEMI[@]}; do
    mris_flatten -w 0 -distances 20 7 ${xh}.full.patch.3d  ${xh}.full.patch.flat
    mris_flatten -w 0 -distances 20 7 ${xh}.occip.patch.3d  ${xh}.occip.patch.flat
done

If everything went well, you now have flatmaps of the white matter surface. 
They should look somewhat like this:  
[FULL]  
![RH_flatmap](pics/RH_Flat_Sulci.png) ![RH_flatmap_gray](pics/RH_Flat_Sulci_gray.png)

In [ ]:
# check your result
tksurfer ${SUBJ} rh inflated -patch rh.full.patch.flat -curv # for a red/green curvature map
#tksurfer ${SUBJ} rh inflated -patch rh.full.patch.flat -gray # for a gray curvature map

#### Fix the pial surface now

In [3]:
freeview -v ${SUBJECTS_DIR}/${SUBJ}/mri/T1.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz \
    -f ${SUBJECTS_DIR}/${SUBJ}/surf/lh.white:edgecolor=yellow \
    ${SUBJECTS_DIR}/${SUBJ}/surf/lh.pial:edgecolor=red \
    ${SUBJECTS_DIR}/${SUBJ}/surf/rh.white:edgecolor=yellow \
    ${SUBJECTS_DIR}/${SUBJ}/surf/rh.pial:edgecolor=red

#### Copy Freesurfer results back to NMT template folder

In [15]:
cp -r ${SUBJECTS_DIR}/${SUBJ}/surf ${fsSurf}/surf

In [ ]:
# return to startpath
cd ${startpath}

## Step 4: Create additional surfaces using Connectome Workbench.
This may not be necessary at all. Try the Freesurfer approach and the conversion to gifti with `mris_convert` first (as explained above)

### Requirements
  - Connectome Workbench (https://www.humanconnectome.org/software/connectome-workbench)  

In [ ]:
# THIS SHOULDN'T BE NECESSARY SINCE WE'RE GETTINGS THESE OUTPUTS ALSO FROM THE FREESURFER PIPELINE !! #

# set path to where you want the results
wbSurf_path=${NMT_path}/single_subject_scans/${SUBJ}/wbSurf

echo 'Extracting surfaces from segmentation'
cd ${wbSurf_path}

# create surfaces
IsoSurface -input ${NMT_ss_path}/NMT_${SUBJ}_process/${SUBJ}_segmentation.nii.gz \
    -isorois -o_gii surf

# rename to something recognizable
mv surf.k2.gii gm.surf.gii # gm surface
mv surf.k3.gii wm.surf.gii # wm surface

# Smooth the surfaces a bit
echo 'Smoothing the surfaces a bit' 
wb_command -surface-smoothing wm.surf.gii 0.5 1 wm_sm.surf.gii
wb_command -surface-smoothing gm.surf.gii 0.5 1 gm_sm.surf.gii

# inflate the surfaces
echo 'Inflating the surfaces'
wb_command -surface-generate-inflated \
    wm_sm.surf.gii infl_wm_sm.surf.gii vinfl_wm_sm.surf.gii
wb_command -surface-generate-inflated \
    gm_sm.surf.gii infl_gm_sm.surf.gii vinfl_gm_sm.surf.gii

# view the result in your favorite viewer
echo 'Inspect the surfaces in a viewer'
freeview -f wm_sm.surf.gii infl_wm_sm.surf.gii vinfl_wm_sm.surf.gii \
    gm_sm.surf.gii infl_gm_sm.surf.gii vinfl_gm_sm.surf.gii